In [ ]:
import pandas as pd

In [ ]:
gene_info = pd.read_csv('/home/jmatsen/Neo4j_meta4/data/gene_info.tsv', sep='\t')

In [ ]:
network = pd.read_csv('/home/jmatsen/Neo4j_meta4/data/network.txt', sep='\t')

In [ ]:
gene_info.shape

In [ ]:
gene_info.head(2)

In [ ]:
# df['date'] = df['raw'].str.extract('(....-..-..)', expand=True)
network['target_organism'] = network['target'].str.extract('([A-z]+[0-9]+)_[0-9]+')
network['target_gene'] = network['target'].str.extract('[A-z]+[0-9]+_([0-9]+)')
network['source_organism'] = network['source'].str.extract('([A-z]+[0-9]+)_[0-9]+')
network['source_gene'] = network['source'].str.extract('[A-z]+[0-9]+_([0-9]+)')

In [ ]:
network.head(2)

In [ ]:
network.reset_index(drop=True, inplace=True)
network.head(2)

In [ ]:
network.to_csv('../data/network_broken.tsv', sep='\t', index=False)

In [ ]:
network.to_csv('../data/network_broken.csv', index=False)

In [ ]:
network.head(20).to_csv('../data/network_broken_20.csv', index=False)

In [ ]:
assert False

In [ ]:
print(network.shape)
joined.shape

In [ ]:
joined.head()

In [ ]:
joined.to_csv('/home/jmatsen/Neo4j_meta4/data/network_with_info.txt')

Trim out just genomes with "ethylo" in them.

In [ ]:
joined.genome.unique()

In [ ]:
#methylo_subset = 
# df[df['A'].str.contains("hello")]
#methylo = 
joined[joined.genome.str.contains('ethylo')]

In [ ]:
print(joined.shape)
print(methylo.shape)